# Insurance Policy Chatbot

**CARGAR LA BASE DE DATOS**

The main objective of this notebook is to guide the reader through the process of understanding how the chatbot works and was build.

The main functionality is to responde to the user's questions about insurance policies based on a PDF database (and WEB search).

The beggining of the process start with the download of the PDFs from AWS S3 into the local storage (_folder:_ `backend/database_s3/queplan_insurance`), then we can start generating our embedding vectors for each of the documents.

In [1]:
# Download the PDF files
from backend.download import download_data

download_data()

ModuleNotFoundError: No module named 'utils'

That was an intended error.

The architecture of the project intend to be converted into a microservice architecture, and so, it is necesary to use the `os` and `sys` libraries to indicate the microservice root path for executing the files.

For that reason, we kindly ask for you to move to the 'injection' directory by running the following command:

`cd injection`

Then, you can run the following command to execute the main file:

- `python A_download_pdf.py`
- `python B_...`

<br />
<br />
<br />

----

This is how you could run the code from here:

In [2]:
import sys, os

service = "backend"
sys.path.insert(0, os.path.join(os.path.dirname('.'), service))

from backend.download import download_data

download_data()

Download queplan_insurance/POL120190177.pdf as c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL120190177.pdf
Download queplan_insurance/POL320130223.pdf as c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL320130223.pdf
Download queplan_insurance/POL320150503.pdf as c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL320150503.pdf
Download queplan_insurance/POL320180100.pdf as c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL320180100.pdf
Download queplan_insurance/POL320190074.pdf as c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL320190074.pdf
Download queplan_insurance/POL320200071.pdf as c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL320200071.pdf
Download queplan_insurance/POL320200214.pdf as c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\quepla